# Wash trade detection
## Prep

In [3]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import networkx as nx

def create_connection(df_file):
    '''create connection to sqlite db
    :param db_file: database file
    :return: connection object
    '''
    conn = None

    try:
        conn = sqlite3.connect(df_file)
    except Error as e:
        print(e)

    return conn

conn = create_connection('nfts.sqlite')
cursor = conn.cursor()
query = '''
SELECT * FROM transfers
'''
transfers_df = pd.read_sql_query(query, conn)

# remove mints from transaction list (not applicable to wash trade detection) 
transfers_clean = transfers_df[transfers_df['to_address'] != '0x0000000000000000000000000000000000000000']

transfers_clean = transfers_clean.reindex()

# get list of all nodes, prepare for mapping
nodes_1 = transfers_clean['from_address'].to_numpy()
nodes_2 = transfers_clean['to_address'].to_numpy()
nodes = np.append(nodes_1, nodes_2)
nodes = set(nodes)

# map original node IDs to numeric for faster computation
nodes_map = {node:indx for indx, node in enumerate(nodes)}

# same as above for token IDs
token_ids = transfers_clean['token_id'].unique() # list of unique token IDs
tokens_map = {token:indx for indx, token in enumerate(token_ids)}

# convert to numpy array for faster computation
transfers_array = transfers_clean.to_numpy() 
# 0:event_id, 1:trans_hash, 2:block_number, 3:nft_add, 4:token_id, 5:from, 6:to, 7:transaction_val, 8:timestamp

# map node_id, token_id strings to numbers using our dict, should just be cleaner
for i in transfers_array: 
    i[5] = nodes_map[i[5]]
    i[6] = nodes_map[i[6]]
    i[4] = tokens_map[i[4]]


## Graph generation for each collection and token, along with strong component detection

In [4]:
# generating graph for each collection and token pairing
token_graphs = {}
for i in transfers_array:
    if (i[3], i[4]) not in token_graphs: #<-- this is where we would add logic to include the address, and token id
        G = nx.MultiDiGraph()
        G.add_weighted_edges_from([(i[5], i[6], i[7])])
        token_graphs[(i[3], i[4])] = {'graph': G}
    else:
        token_graphs[(i[3], i[4])]['graph'].add_weighted_edges_from([(i[5], i[6], i[7])])

# for each graph, running strong component algorithm and tallying risky transactions/tokens
for i in token_graphs:
    all_components = nx.strongly_connected_components(token_graphs[i]['graph'])
    token_graphs[i]['strong components'] = [i for i in list(all_components) if len(i) == 2]

    # tally risky transactions for each token
    if token_graphs[i]['strong components'] == []: # if no strong components
        token_graphs[i]['risky trans'] = 0
        token_graphs[i]['risky tokens'] = 0
    else:
        for edge in token_graphs[i]['strong components']:
            src, targ = tuple(edge)
            all_edge_data = [token_graphs[i]['graph'].get_edge_data(src, targ), token_graphs[i]['graph'].get_edge_data(targ, src)]
            
            if 'risky trans' not in token_graphs[i]:
                token_graphs[i]['risky trans'] = sum(len(k) for k in all_edge_data)
            else:
                token_graphs[i]['risky trans'] += sum(len(k) for k in all_edge_data)
            
        token_graphs[i]['risky tokens'] = 1


## IDing self-trades and adding to risky token/transaction counts

In [5]:
self_transfers_array = transfers_array[transfers_array[:,5] == transfers_array[:,6]]

# tallying self transfers/transactions as wash trading, adding to our risky trans count for each token
for i in self_transfers_array:
    if 'risky trans' not in token_graphs[(i[3], i[4])]:
        token_graphs[(i[3], i[4])]['risky trans'] = 1
    else:
        token_graphs[(i[3], i[4])]['risky trans'] += 1 

## Aggregating risky business at the collection level

In [ ]:
filtered_collection_dict = {i:{'risky trans': token_graphs[i]['risky trans'], 'risky tokens': token_graphs[i]['risky tokens']} for i in token_graphs}
new_df = pd.DataFrame.from_dict(filtered_collection_dict, orient='index')
risky_trans_grouped = new_df.groupby(level=0).sum('risky trans').reset_index().rename(columns={'index': 'nft_address'})

# query DB to get total transactions, tokens by collection
totals_query = '''
SELECT transactions.nft_address, COUNT(*)  AS trans_counts, toke_table.token_counts
FROM transfers transactions
JOIN  (
	SELECT nft_address,  COUNT(DISTINCT token_id ) AS token_counts
	FROM transfers
	GROUP BY nft_address) toke_table ON toke_table.nft_address = transactions.nft_address
GROUP BY transactions.nft_address
'''
totals_df = pd.read_sql_query(totals_query, conn)

combined_df = pd.merge(totals_df.assign(x=totals_df.nft_address.astype(str)), \
    risky_trans_grouped.assign(x=risky_trans_grouped.nft_address.astype(str)), \
    how='left', on='x')

# cast nulls as 0 after joining totals and risky tables
combined_df['risky trans'].fillna(0, inplace=True)
combined_df['risky tokens'].fillna(0, inplace=True)

combined_df.drop(['x', 'nft_address_y'], axis=1, inplace=True)

# calculate new columns
combined_df['risky_trans_ratio'] = combined_df['risky trans'] / combined_df['trans_counts']
combined_df['risky_token_ratio'] = combined_df['risky tokens'] / combined_df['token_counts']

combined_df.rename(columns={'nft_address_x':'nft_address',\
    'trans_counts':'total_transactions',\
    'token_counts':'total_tokens',
    'risky trans':'risky_transactions',
    'risky tokens':'risky_tokens',
    'risky_trans_ratio':'risky_transaction_ratio'}, inplace=True)

In [ ]:
# write wash trade results to csv
#pd.DataFrame.to_csv(combined_df, 'data/collection_wash_trades.csv')